#  Predicting Cadets Likely to Struggle in Physics I

While the United States Coast Guard Academy (USCGA) recruits high-achieving students from across the nation and around the globe, every year we have cadets who struggle to pass Physics I.  Failure to pass Physics has many undesirable consequences, including lowering a cadet's GPA, potentially delaying graduation and deployment to the operational fleet, and possibly even leading to a cadet's disenrollment from the Academy.  It is in the best interests of both the United States Coast Guard and the cadets themselves to do whatever we can to help cadets to pass Physics I the first time they take it.  Early intervention is vital, and academic support programs are in place to help struggling cadets.  However, at-risk cadets are difficult to identify until they have already failed an exam or two, at which point they are already in distress.  This tool seeks to mitigate that problem by predicting - right at the start of the semester - the cadets most at risk to have an unfavorable outcome in Physics I (defined here as earning a final grade of C- or worse), for the purpose of getting them into an academic support program at the earliest sign of trouble.

## Model Training

This tool is a ML model that identifies cadets most at risk to struggle in Physics I based on their prior performance at the USCGA and in high school.  Both the training and prediction data require access to cadet PII, including grades and GPAs.  Consequently, the tool is presented as a Jupyter notebook for use by the USCGA Vice Provost for Academic Affairs or their designee (e.g. the Physics I Course Coordinator).

In it's current form, the model has been trained on cadet data for Class Years 2020-2025.  You should not retrain the model unless you wish to update it based on the performance of classes since then.  If you only want to identify at-risk cadets fot the upcoming semester, proceed to the "Predictions" section.

### Import and merge training data, import target data

First, we'll import the institutional training data.  The current code assumes the cadet data will be presented as a .csv file with 15 fields in each row. The first field is a unique identifier (i.e. Cadet Code or equivalent).  The other fields and their data types are:
* Gender (string)
* Academic Class Year  (integer)  
* Ethnicity (string)
* Math Placement (integer, CGA math class number) 
* Most recent CGA GPA (decimal)
* HS GPA (decimal)
* HS GPA Scale (4.0, 100, etc.) (decimal)
* Standardized Test Used (SAT or ACT) (string) 
* Math Score (SAT equivalent) (integer)  
* Verbal Score (SAT equivalent) (integer)  
* Times enrolled in Calculus I (integer) 
* Calculus 1 Last Grade (letter) 
* Calculus I Last Quality Points (decimal)
* Times enrolled in Physics I (integer, 1 unless repeating)  

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
import numpy as np
import pandas as pd

train_instit = pd.read_csv("./Train-instit.csv", index_col=0)

print(type(train_instit))
train_instit.shape

In [ ]:
train_instit.info()

The provided institutional data includes the demographic information "Gender" and "Ethnicity".  We do not want these categories to bias our results, so we'll drop them.  We'll also drop the Class Year (not predictive) and "PhysicsITimes", as we'll bring that in with the departmental data.

In [ ]:
train_instit.drop(columns=['Gender', 'AcademicClassYr', 'Ethnicity', 'PhysicsITimes'], inplace=True)

In [ ]:
train_instit['MathPlacement'] = train_instit['MathPlacement'].astype(str) #Set MathPlacement (class number) as string

Now we'll import departmental data.  This also starts with a unique identifier for each cadet that matches those in the institutional data.  The other fields are "Phys1Grade" (a cadet's Physics 1 final grade for each attempt) and "Physics1Times".  Note that REGIS (the CGA academic record system) may provide only the most recent value for Physics1Times, so the data for cadets repeating Physics 1 in any semester had to be manually updated to match attempt instances with their final grades for each attempt.  Future updates to the training data should be ssimilarly pre-processed.

In [ ]:
train_dept = pd.read_csv("./Train-dept-v2.csv", index_col=0)

print(type(train_dept))
train_dept.shape

In [ ]:
train_df=train_dept.merge(train_instit, on='RecordNum', how='left')

In [ ]:
train_df.info()

Let's find the mean of the standardized test scores "MaxMath" and "MaxVerbal" in case we need to fill any missing values in future cohorts.

In [ ]:
SAT_Math_avg = int(train_df.MaxMath.mean())
SAT_Verbal_avg = int(train_df.MaxVerbal.mean())
print(SAT_Math_avg, SAT_Verbal_avg)

### Data Cleaning

We dropped drop the 'Gender' and 'Ethnicity' columns during data import, on the grounds that these may bias our results.  We also dropped the "AcademicClassYr" column, since year-on-year performance should be roughly similar.  We dropped them from the prediction set as well.  We'll use "Calc1QP" as the Calc 1 performance indicator, so we'll drop "Calc1LastGrade" too.

In [ ]:
train_inst=train_df.drop(columns=['Calc1LastGrade'])

In [ ]:
train_inst.info()

There appear to be 5 missing values in the "LastGPA" column.  These are likely due to disenrollments at the beginning of the semester, and should be dropped.

In [ ]:
train_inst.dropna(subset=['LastGPA'], inplace=True)

About 300 records are missing values for 'Calc1LastGrade' and 'Calc1LastQP'.  This could be because their math placement exam placed them into a class higher than Calc 1, or because they hadn't yet taken Calc 1.  If the former, their effective Calc 1 grade should be 'A' and their Calc 1 Quality Points should be 4.0.  For those cadets, their effective Calc1Number should also be set to 1.  For the others, their QP should be 0.0.

In [ ]:
train_inst['Calc1LastQP'] = train_inst.apply(lambda row: 
                                             0.0 
                                             if pd.isnull(row['Calc1LastQP']) and int(row['MathPlacement']) <= 3111
                                             else row['Calc1LastQP'], axis=1)

train_inst['Calc1LastQP'] = train_inst.apply(lambda row: 
                                             4.0 
                                             if pd.isnull(row['Calc1LastQP']) and int(row['MathPlacement']) > 3111
                                             else row['Calc1LastQP'], axis=1)

train_inst['Calc1Number'] = train_inst.apply(lambda row: 
                                             1 
                                             if int(row['MathPlacement']) > 3111
                                             else row['Calc1Number'], axis=1)

Let's get the HS GPAs on the same scale.

In [ ]:
train_inst['HS_Gpct'] = train_inst.HSGPA / train_inst.MaxHS_GPA_Scale

In [ ]:
train_inst.HS_Gpct.replace([np.inf, -np.inf], np.nan, inplace=True)  #taking care of some pesky inf values

About 300 cadets in our training data don't have HS GPAs listed!  For now, we'll fill them with their Academy GPA.

In [ ]:
train_inst['HS_Gpct'] = train_inst.apply(lambda row: 
                                             row['LastGPA']/4.0 
                                             if pd.isnull(row['HS_Gpct'])
                                             else row['HS_Gpct'], axis=1)

Finally, we need to handle the Physics 1 grades (the thing we're predicting).

Many Physics 1 grades (the target of our predictions) are missing from the training data.  In many cases, these are due to "validation": i.e., testing out of Physics 1.  These cadets almost universally placed into Calc 1 or higher.  For now, we'll assume that any missing Physics 1 grade where the cadet was placed into MATH 3211 was a validation.

In [ ]:
train_inst['Phys1Grade'] = train_inst.apply(lambda row: 
                                             'V' 
                                             if pd.isnull(row['Phys1Grade']) and int(row['MathPlacement']) >= 3111
                                             else row['Phys1Grade'], axis=1)

In [ ]:
train_inst.Phys1Grade.value_counts()

In [ ]:
train_inst.info()

Grades of W or S are usually due to extenuating circumstances.  We'll drop them, along with the 7 rows still missing grades.

In [ ]:
valid_grades = ['V', 'H', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D', 'F']

In [ ]:
df = train_inst[train_inst.Phys1Grade.isin(valid_grades) == True]

In [ ]:
df.Phys1Grade.value_counts()

Note that the classes here are HIGHLY unbalanced.  Only about 1 in 15 cadets earns a grade of C- or worse.  We will beed to address this in the model training section.

In [ ]:
df.info()

As a last step, we will convert our Physics 1 grades into our target labels.  Grades of C- or higher are "successful", while those below are not.

In [ ]:
#Successful_grades = ['V', 'H', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C']  #moved C- to the At Risk category to help balance classes

In [ ]:
def GradeConverter(row):
    successful_grades = ['V', 'H', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C']
    
    if row['Phys1Grade'] in successful_grades:
        outcome = 0
    else:
        outcome = 1
        
    return outcome    

In [ ]:
df['Phys1_outcome'] = df.apply(lambda row: GradeConverter(row), axis=1)

In [ ]:
df.Phys1_outcome.value_counts()

In [ ]:
model_input = df.copy()

In [ ]:
features_to_use = ['MathPlacement','HS_Gpct', 'MaxMath', 'MaxVerbal', 'LastGPA', 'Calc1Number', 'Calc1LastQP', 'PhysicsITimes', 'P1_PrevGrade']

X_m = model_input[features_to_use]
y_m = model_input['Phys1_outcome']

In [ ]:
X_m.head()

In [ ]:
y_m.head()

Now we'll encode the columns in preparation for sampling and classification

We'll need to select and transform the columns we want to feed into our predictor.  MathPlacement is categorical and will need to be OneHotEncoded.  We'll also keep HS_Gpct, LastGPA, MaxMath, MaxVerbal, Calc1Number, Calc1LastQP, and PhysicsITimes.  Phys1_outcome is what we'll be trying to predict.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

columns_to_encode = ['MathPlacement', 'P1_PrevGrade']
ohe = OneHotEncoder()

ohe_trans = ColumnTransformer([
    ('OHE', ohe, columns_to_encode)], 
    remainder='passthrough')

X_encoded = ohe_trans.fit_transform(X_m)
Xenc_cols = ohe_trans.get_feature_names_out()

In [ ]:
X_encoded.shape

In [ ]:
Xenc_cols

In [ ]:
X=pd.DataFrame(X_encoded, columns=Xenc_cols)

In [ ]:
X.info()

In [ ]:
columns_in = list(X.columns)
columns_in

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y = y_m

In [ ]:
y.head()

In [ ]:
import pickle

X.to_pickle('Cap_cleaned_X.pkl')
y.to_pickle("Cap_cleaned_y.pkl")

### Transform and training

Now we're ready to train our model.  Let's start by re-loading the cleaned data and labels.

In [ ]:
X=pd.read_pickle('Cap_cleaned_X.pkl')
y=pd.read_pickle('Cap_cleaned_y.pkl')

In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.metrics import cohen_kappa_score, make_scorer

To train our model and evaluate goodness of redults, we first randomly split the data into training and validation sets.  Then since the classes are highly unbalanced, we perform synthetic minority oversampling using BorderlineSMOTE to balance the classes. In the first implementation of this tool, we then used GridSearchCV to tune the regularization parameter C of the logistic regressor we use for classification.  However, we found that the value of C varied greatly between runs, sometimes by a couple of orders of magnitude.  To address this, we wrapped out gridsearch in a function that loops over the gridsearch 1000 times, and returns the median value for C.  Note:  Do NOT, rerun this optimizer unless you have a couple of hours to spare...

In [ ]:
def Logistic_GS_optimizer(Xin, yin, num_loops):
    
    best_C_list = []
    
    for loop in range(num_loops):
        
                
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
        
        if loop % 50 == 0:
            print("Search loop:", loop)

        smote = BorderlineSMOTE(sampling_strategy='auto')
        smox, smoy = smote.fit_resample(X_train, y_train)
        
        from sklearn.metrics import cohen_kappa_score, make_scorer

        kappa_scorer=make_scorer(cohen_kappa_score)

        log_reg = LogisticRegression(max_iter=1000)
        log_params = {'C': np.logspace(-3, 4, 180)}
        log_gridsrch = GridSearchCV(log_reg, log_params, cv=10, n_jobs=-1, scoring=kappa_scorer)
            
        search_pipe = Pipeline([('scaler', StandardScaler()),
                       ('logreg', log_gridsrch)])
        search_pipe.fit(smox, smoy)
        
        best_C_loop = log_gridsrch.best_params_['C']
        
        if loop % 50 == 0:
            print(f'Best C for loop {loop} = {best_C_loop}')
        
        best_C_list.append(best_C_loop)
        
    return best_C_list

In [ ]:
# - CAUTION - uncomment to rerun the gridsearch optimizer. - CAUTION - LONG RUN TIME
#capstone_C = Logistic_GS_optimizer(X, y, 1000)

# the histogram of the C values
sns.histplot(data=capstone_C, bins=25, log_scale=True)
plt.xlabel("Regularization parameter (C) best value", size=14)
plt.ylabel("Count", size=14)
plt.axvline(x=np.median(capstone_C),
            color='blue', ls='--', lw=2.5)

In [ ]:
plt.savefig("Seaborn_displot_histogram_with_median_line_Python.png")

Based on 1000 loops over our grid search, the median (and likely best) value for the regularization parameter C is about 0.65.  I have hardcoded C below so this notebook can be run for demonstration.

In [ ]:
#NOTE: C hard=coded to prevent needing to rerun the hyperparameter fitter above
#best_C = np.median(capstone_C)
best_C = 0.6540974531071987
print(best_C)

Now we'll actually train the model.  We'll start by splitting the data into train and test sets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

print("Shape of the complete data set:", X.shape)
print("Shape of the train data set:", X_train.shape)
print("Shape of the test data set:", X_test.shape)

Next, perform oversampling to balance the classes.

In [ ]:
smote = BorderlineSMOTE(sampling_strategy='auto')
smox, smoy = smote.fit_resample(X_train, y_train)

In [ ]:
smoy.value_counts()

...and train the model.  Note that we're pickling the results so we can reload the trained model for prediction of later cohorts.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

result_pipe = Pipeline([('scaler', StandardScaler()),
                       ('classifier', LogisticRegression(max_iter=1000, C=best_C))]) 
                        
result_pipe.fit(smox.values, smoy.values)

with open('trained-model-physics1.pkl', 'wb') as f:
    pickle.dump(result_pipe, f)

ypred = result_pipe.predict(X_test.values)
print(classification_report(y_test, ypred))
pd.DataFrame(confusion_matrix(y_test, ypred),
             index = ['Successful actual', 'At Risk actual'],
             columns = ['Successful pred', 'At Risk pred'])

Above is the confusion matrix for the particular train/test split from the last time the model was fit.  There are probably about 50 cadets predicted to be at risk of earning a C- or worse in the test set.  Of these, less than 20 actually earned a grade of C- or lower - we have many "false positives" ("positive" in this case meaning they are predicted to be at risk).  This is actually a good thing!  History is not destiny, and most cadets work hard and pass Physics I, regardless of their prior performance.  The ones we're after here are the ones MOST likely to have a hard time.  Let's see if we can identify them.

### Model Results

We'll start by retrieving the model probabilities.  A probability of 1.0 means the model thinks a cadet is almost certain to ge a C- or woese.

In [ ]:
probs=result_pipe.predict_proba(X_test.values)

We can use these probabilityes to evaluate how well our model did.  Here's the precision-recall curve for this run.

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay, RocCurveDisplay

y_preddisp = result_pipe.predict_proba(X_test.values)[:, 1]
PrecisionRecallDisplay.from_predictions(y_test.values, y_preddisp)

plt.show()


There is not as much area under that curve as we'd normally like to see.  The model has a hard time clearly separating "At-risk" cadets from those likely to succeed.  History is not destiny.  

In [ ]:
y_out_df = pd.DataFrame(np.array(y_test), columns=['actual'], index=X_test.index)
y_out_df['predicted']=ypred
y_out_df['probability']=probs[:, 1]
y_out_df.head(20)

What about Cohen's kappa?  According to Wikipedia, Cohen's kappa measures the agreement between two raters who each classify N items into C mutually exclusive categories.

In [ ]:
lr_kappa = cohen_kappa_score(ypred, y_test)
print(lr_kappa)

A Cohen's kappa of 0.4 indicates "fair to moderate" agreement between our "raters" - one of which is the predicted outcomes, and the other the actual outcomes.

The ROC curve provides another view.  Let's plot it.

In [ ]:
y_probs = pd.DataFrame(result_pipe.predict_proba(X_test.values)[:, 1])

In [ ]:
prob_at_risk = probs[:,1]
prob_at_risk[:22]

In [ ]:
RocCurveDisplay.from_predictions(y_test, y_preddisp)
plt.show()

Again, the false positive rate creeps up as we approach true positive rates of 90%.  The model has difficulty cleanly separating the Successful and At-risk cadets.  Let's look at how many cadets fall into each probability range.

In [ ]:
round_prob = [round(num, 3) for num in prob_at_risk]

In [ ]:
# the histogram of the probabilities
sns.histplot(data=round_prob, bins=20, log_scale=(False, True))

Most hadets are not at risk of dooing poorly in Physics 1 based solely on prior performance.  They fall at the left of this probability histogram.  (Note that the counts in each probability bin are plotted logarithmically.)  Rther than clean separation, or at least having a minimum at 0.5, the natural break appears to be closer to 0.7.  Again, many "at-risk" cadets work hard and do fine.  Let's see if we can narrow things down by seeing if our model precision improves if we set a cutoff at progressively higher probabilities.

In [ ]:
y_out_df.loc[y_out_df['probability'] >= .96]

In [ ]:
from sklearn.metrics import accuracy_score
threshold_accuracy=accuracy_score(y_out_df.loc[y_out_df['probability'] >= .96].actual, y_out_df.loc[y_out_df['probability'] >= .96].predicted)

In [ ]:
print(threshold_accuracy)

In [ ]:
thresholds = 1.5 - np.geomspace(1, .51, num=30)

In [ ]:
accuracies = []

for num in thresholds:
    accuracies.append(accuracy_score(y_out_df.loc[y_out_df['probability'] > num].actual,
                                     y_out_df.loc[y_out_df['probability'] > num].predicted))

In [ ]:
plt.plot(thresholds, accuracies)
plt.xlabel("Threshold probability", size=14)
plt.ylabel('"At-risk" precision', size=14)
plt.title('Prediction precision at different probability thresholds')
plt.axvline(x=0.96, color='red', ls='--', lw=2.5)
plt.show
plt.savefig("Matplotlib_probability_thresholds_w_line.png")

Here we've plotted the precision of the model's "At-risk" prediction vs. the cadet's probability of being at risk.  Precision increases slowly below at-risk probabilities of 0.9, then increases dramatically.  By the time probabiity reaches 0.96, precision is above 80%.  THESE are the cadets we want to make sure we identify, and possibly assign to academic support right at the start of the semester.  We can print out a list, by identifier:

In [ ]:
y_out_df.loc[y_out_df['probability'] >= .96]

Finally, let's see if we can fet some insight into WHY the model thinks they might struggle.  The Locally Interpretable, Model-Agnostic Explainer (LIME) perturbs each data point to see which factors are most significant in the model's prediction.  LIME also provides an Explainer to present its findings graphically.

In [ ]:
import lime
import lime.lime_tabular

X_features = ['MATH_3107', 'MATH_3111', 'MATH_3115', 'MATH_3117', 'MATH_3211', 'MATH_3213', 'MATH_3215', 
              'Phys_Prev_Grade_F', 'Phys_Prev_Grade_N', 'HS_GPA', 'SAT_Math', 'SAT_Verbal', 'CGA_GPA', 'Calc_1_times', 'Calc_1_QP', 'Phys_1_times' ]

explainer = lime.lime_tabular.LimeTabularExplainer(np.array(smox),
                                                   feature_names=X_features,
                                                   training_labels=smoy,
                                                   class_names=['Likely successful', 'Likely At-risk'],
                                                   # categorical_features=,
                                                   # There is no categorical features in this example, otherwise specify them.
                                                   verbose=True, mode='classification')

In [ ]:
report_at_risk = list(y_out_df.loc[y_out_df['probability'] >= .96].index)

In [ ]:
for instance in report_at_risk:
    print(f'Unique ID: {instance}')
    exp = explainer.explain_instance(X_test.loc[instance].values, result_pipe.predict_proba, num_features=6)
    exp.show_in_notebook(show_table=True, show_all=False)

According to LIME, the cadets most at-risk to struggle in Physics 1 are usually those with a low GPA at the Academy, regardless of earlier GPA, standardided test scores (which rarely appear), or other factors.  Poor performance in Calc I or placement into Math 3107 are also frequently in the top reasons a cadet is likely at risk.  These explanations may be useful in targeting appropriate inteventions.

## Predict at-risk cadets for a new cohort

To make predictions on a new cohort of Physics I cadets, we'll start by importing their institutional data.  The data format should be a .CSV file with the fields described in the "Model Training" section above. We dropped drop the 'Gender' and 'Ethnicity' columns during training data import, on the grounds that these may bias our results. We also dropped the "AcademicClassYr" column, since year-on-year performance should be roughly similar. We'll drop them from the prediction set as well.  We'll likewise drop the "Calc1Grade" column, since we'll use each cadet's Calc 1 Quality Points as the performance indicator.

In [ ]:
pred_df = pd.read_csv("./pred-inst.csv", index_col=0)

pred_df.drop(columns=['Gender', 'AcademicClassYr', 'Ethnicity'], inplace=True)
pred_df['MathPlacement'] = pred_df['MathPlacement'].astype(str) #Set MathPlacement (class number) as string
pred_inst = pred_df.drop(columns=['Calc1LastGrade'])

pred_df.shape

In the training data, bout 300 records were missing values for 'Calc1LastGrade' and 'Calc1LastQP'. This could be because their math placement exam placed them into a class higher than Calc 1, or because they hadn't yet taken Calc 1. If the former, their effective Calc 1 grade should be 'A' and their Calc 1 Quality Points should be 4.0. For those cadets, their effective Calc1Number was set to 1. For the others, their QP was set to 0.0.  We'll apply the same replacement to the prediction data.

In [ ]:
pred_inst['Calc1LastQP'] = pred_inst.apply(lambda row: 
                                             0.0 
                                             if pd.isnull(row['Calc1LastQP']) and int(row['MathPlacement']) <= 3111
                                             else row['Calc1LastQP'], axis=1)

pred_inst['Calc1LastQP'] = pred_inst.apply(lambda row: 
                                             4.0 
                                             if pd.isnull(row['Calc1LastQP']) and int(row['MathPlacement']) > 3111
                                             else row['Calc1LastQP'], axis=1)

pred_inst['Calc1Number'] = pred_inst.apply(lambda row: 
                                             1 
                                             if int(row['MathPlacement']) > 3111
                                             else row['Calc1Number'], axis=1)

We'll get all of the HS GPAs on the same scale...

In [ ]:
pred_inst['HS_Gpct'] = pred_inst.HSGPA / pred_inst.MaxHS_GPA_Scale
pred_inst.HS_Gpct.replace([np.inf, -np.inf], np.nan, inplace=True)
pred_inst['HS_Gpct'] = pred_inst.apply(lambda row: 
                                             row['LastGPA'] /4.0
                                             if pd.isnull(row['HS_Gpct'])
                                             else row['HS_Gpct'], axis=1)

In [ ]:
pred_inst.info()

...and fill any missing GPAs with their CGA GPA.

In [ ]:
pred_inst.HS_Gpct.replace([np.inf, -np.inf], np.nan, inplace=True)  #taking care of some pesky inf values

In [ ]:
pred_inst['HS_Gpct'] = pred_inst.apply(lambda row: 
                                             row['LastGPA']/4.0 
                                             if pd.isnull(row['HS_Gpct'])
                                             else row['HS_Gpct'], axis=1)

In [ ]:
pred_inst.head()

For the class years 2020-2025, the mean SAT Math score was 655, and the mean SAT Verbal Score was 652.  In the training results, standardized test scores were never strongly predictive of the cadets most-at-risk.  For prediction purposes, we'll fill any missing standardized test scores (reported as 0 in the input .csv file) with the corresponding mean score.

In [ ]:
SAT_Math_avg = 655
SAT_Verbal_avg = 652

pred_inst['MaxMath'].replace(0, 655, inplace=True)
pred_inst['MaxVerbal'].replace(0, 652, inplace=True)
pred_inst.head()

In [ ]:
def prevgrade(row):
    
    if row['PhysicsITimes'] == 0:
        outcome = 'N'
    else:
        outcome = 'F'
        
    return outcome    

In [ ]:
pred_inst['P1_PrevGrade'] = pred_inst.apply(lambda row: prevgrade(row), axis=1)

In [ ]:
pred_inst.head()

In [ ]:
pred_input = pred_inst.copy()

In [ ]:
features_to_use = ['MathPlacement','HS_Gpct', 'MaxMath', 'MaxVerbal', 'LastGPA', 'Calc1Number', 
                   'Calc1LastQP', 'PhysicsITimes', 'P1_PrevGrade']
pred_X_m = pred_input[features_to_use]

Now we'll encode the columns in preparation for sampling and classification.  As we did for the training data, we'll need to select and transform the columns we want to feed into our trained model for prediction. MathPlacement is categorical and will need to be OneHotEncoded. We'll also keep HS_Gpct, LastGPA, MaxMath, MaxVerbal, Calc1Number, Calc1LastQP, and PhysicsITimes. We'll use to predict the likely outcome (Physics 1 Successful vs. Unsuccessful).

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

columns_to_encode = ['MathPlacement', 'P1_PrevGrade']
ohe = OneHotEncoder()

ohe_trans = ColumnTransformer([
    ('OHE', ohe, columns_to_encode)], 
    remainder='passthrough')

pred_X_encoded = ohe_trans.fit_transform(pred_X_m)
pred_Xenc_cols = ohe_trans.get_feature_names_out()

In [ ]:
pred_X=pd.DataFrame(pred_X_encoded, columns=pred_Xenc_cols)
pred_X.info()

In [ ]:
trained_columns_in = ['OHE__MathPlacement_3107', 'OHE__MathPlacement_3111','OHE__MathPlacement_3115',
                      'OHE__MathPlacement_3117', 'OHE__MathPlacement_3211','OHE__MathPlacement_3213',
                      'OHE__MathPlacement_3215', 'OHE__P1_PrevGrade_F', 'OHE__P1_PrevGrade_N', 'remainder__HS_Gpct',
                      'remainder__MaxMath', 'remainder__MaxVerbal', 'remainder__LastGPA', 'remainder__Calc1Number',
                      'remainder__Calc1LastQP','remainder__PhysicsITimes']
for column_name in trained_columns_in:
    if column_name not in pred_X.columns:
        pred_X[column_name] = 0
        
pred_X = pred_X.reindex(columns=trained_columns_in)

In [ ]:
pred_X.info()

In [ ]:
import pickle

pred_X.to_pickle('pred_cleaned_X.pkl')

Let's import what we need for the predictions.

In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.metrics import cohen_kappa_score, make_scorer

In [ ]:
#NOTE: C hard=coded to prevent needing to rerun the hyperparameter fitter above
best_C = 0.6540974531071987
print(best_C)

Now we'll load the trained pipeline that we'll use to predict outcomes.

In [ ]:
with open('trained-model-physics1.pkl', 'wb') as f:
    loaded_model = pickle.dump(result_pipe, f)

In [ ]:
cohort_pred = result_pipe.predict(pred_X.values)
cohort_probs=result_pipe.predict_proba(pred_X.values)

In [ ]:
cohort_out_df = pd.DataFrame(cohort_pred, columns=['predicted_at_risk'], index=pred_X.index)
#cohort_out_df['predicted']=ypred
cohort_out_df['probability']=cohort_probs[:, 1]
cohort_out_df.head(20)

The model has predicted the probable outcome in Physics I for each cadet, based solely on their prior performance.  As noted above, prior performance is not the only predictor of future performance; what a cadet actually does during the semester is of far greater importance.  Most cadets predicted to struggle in Physics I will work hard and succeed.  However, those most likely to be at risk may need extra support and should be placed in academic support at the earliest sign of difficulty.  Let's identify those most at risk, using the 96% threshold from training.

In [ ]:
cohort_out_df.loc[cohort_out_df['probability'] >= .96]

For the cohort in Physics I at the time this tool was developed, 11 cadets were predicted to be most-at-risk at the beginning of the semester.  Let's generate At-Risk Reports for those cadets.

In [ ]:
import lime
import lime.lime_tabular

X_features = ['MATH_3107', 'MATH_3111', 'MATH_3115', 'MATH_3117', 'MATH_3211', 'MATH_3213', 'MATH_3215', 
              'Phys_Prev_Grade_F', 'Phys_Prev_Grade_N', 'HS_GPA', 'SAT_Math', 'SAT_Verbal', 'CGA_GPA', 'Calc_1_times', 'Calc_1_QP', 'Phys_1_times' ]

explainer = lime.lime_tabular.LimeTabularExplainer(np.array(pred_X.values),
                                                   feature_names=X_features,
                                                   training_labels=cohort_pred,
                                                   class_names=['Likely successful', 'Likely At-risk'],
                                                   # categorical_features=,
                                                   # There is no categorical features in this example, otherwise specify them.
                                                   verbose=True, mode='classification')

In [ ]:
report_at_risk = list(cohort_out_df.loc[cohort_out_df['probability'] >= .96].index)

In [ ]:
for instance in report_at_risk:
    print(f'Unique ID: {instance}')
    exp = explainer.explain_instance(pred_X.loc[instance].values, result_pipe.predict_proba, num_features=6)
    exp.show_in_notebook(show_table=True, show_all=False)

These results can now be used to monitor at-risk cadets, or to fill academic support programs for the first weeks of the semester.